In [ ]:
# Group is generated by x and y
# Only relation is x^{N_x} = y^{N_y}
N_x = 2
N_y = 3

# Shrinking factor when going from x to y
shrinking_x = 2/(N_x+1)

# Shrinking factor when going from y to x
shrinking_y = 2/(N_y+1)

# Point are considered equal if they are equal up to this many decimals
decimal_places = 5

resolution = 10

# Helix of radius r, with starting point at angle alpha0
# Returns position of helix at angle alpha0+alpha
def arrow(starting_point,r,alpha,alpha0):
    x0,y0,z0 = starting_point[0], starting_point[1], starting_point[2]
    x = x0 + r*np.cos(alpha0+alpha)-r*np.cos(alpha0)
    y = y0 + r*np.sin(alpha0+alpha)-r*np.sin(alpha0)
    z = z0 + alpha
    return (x,y,z)

# Draw action of generator x in Cayley graph
# Follows helix from angle0 to alpha0+2pi/3
def draw_action_x(starting_point,r,alpha0):
    theta = np.linspace(0, 2 * np.pi/N_x, resolution)
    (x,y,z) = arrow(starting_point,r,theta,alpha0)
    ax.plot(x,y,z, label='parametric curve')
    return (arrow(starting_point,r,2*np.pi/N_x,alpha0),alpha0+2*np.pi/N_x)

# Draw action of generator y in Cayley graph
# Follows helix from angle0 to alpha0+pi
def draw_action_y(starting_point,r,alpha0):
    theta = np.linspace(0,2*np.pi/N_y, resolution)
    (x,y,z) = arrow(starting_point,r,theta,alpha0)
    ax.plot(x,y,z, label='parametric curve')
    return (arrow(starting_point,r,2*np.pi/N_y,alpha0),alpha0+2*np.pi/N_y)

# Start drawing Cayley graph
# First draw action of x at angle angle0 and y at angle alpha0+pi
# Then restart at endpoint of x-action
# And restart at endpoint of y-action, with swapped angles
def start_drawing(starting_point,x_radiuses,y_radiuses,alpha0,depth):
    r_x = x_radiuses[XY(starting_point)]
    (new_point_x,alpha0_x) = draw_action_x(starting_point,r_x,alpha0)
    x_radiuses[XY(new_point_x)] = r_x
    if not(XY(new_point_x) in y_radiuses):
        y_radiuses[XY(new_point_x)] = round(r_x*shrinking_x,decimal_places)
    
    r_y = y_radiuses[XY(starting_point)]
    (new_point_y,alpha0_y) = draw_action_y(starting_point,r_y,alpha0+np.pi)
    y_radiuses[XY(new_point_y)] = r_y
    if not(XY(new_point_y) in x_radiuses):
        x_radiuses[XY(new_point_y)] = round(r_y*shrinking_y,decimal_places)
    
    if depth >= 1:
        start_drawing(new_point_x,x_radiuses,y_radiuses,alpha0_x,depth-1)
        start_drawing(new_point_y,x_radiuses,y_radiuses,alpha0_y+np.pi,depth-1)
    return

# To see which helix you are on, you only need the X and Y coordinates
# up to a certain number of decimal places
def XY(point):
    x = round(point[0],decimal_places)
    y = round(point[1],decimal_places)
    return (x,y)

# Enable interactive 3d plot
%matplotlib notebook

import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt

mpl.rcParams['xtick.labelsize'] = 0    # Remove labels
fig = plt.figure()
ax = fig.gca(projection='3d')
ax._axis3don = False                   # Remove axes

start = (2,0,0)
x_radiuses = {XY(start) : 2}
y_radiuses = {XY(start) : 2*shrinking_x}

# number of arrows drawn = 2^(depth+1)
depth = 6      

start_drawing(start,x_radiuses,y_radiuses,0,depth)

plt.show